<a href="https://colab.research.google.com/github/shardul28/political-stance-detection/blob/Mscproject/MODEL_80%25_%20acc_%20decent_recall.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Imports

In [ ]:
!pip install mit-news-classify


In [ ]:
import nltk
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [ ]:
import pandas as pd
import re
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Embedding
from tensorflow.keras.optimizers import Adam
import pickle
from mitnewsclassify import tfidf as mitx
from nltk.corpus import stopwords
from tqdm import tqdm
from nltk.util import ngrams
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer,PorterStemmer
from sklearn.feature_extraction.text import CountVectorizer
from collections import defaultdict,Counter
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
cd /content/drive/MyDrive/project_data

/content/drive/MyDrive/project_data


# MODEL


In [ ]:
infile = open('/content/drive/MyDrive/project_data/clean_data.pkl','rb')
df = pickle.load(infile)

In [ ]:
df.head()

,text,url,title,content,bias,retweet_count,favorite_count,source_name,source,clean_content,text_lemmatized,clean_text_lemmatized,clean_content_with_Stopwords
0,"Army Officer, 4 Others Killed In Encounter In ...",https://www.ndtv.com/india-news/army-officer-4...,"Army Officer, 4 Personnel Killed In Encounter ...",an army officer and four other personnel were ...,3,9.0,63.0,ndtv,0,army officer personnel killed action encounter...,"[army, officer, personnel, killed, action, enc...","[army, officer, personnel, killed, action, enc...",an army officer and four other personnel were ...
1,"#AryanKhan Case: No Bail For Him Today, Next H...",https://ndtv.com/india-news/mumbai-drug-bust-c...,"Aryan Khan Case: No Bail For Him Today, Next H...",aryan khan did not get bail today for the thir...,3,5.0,44.0,ndtv,0,aryan khan bail arrest week ago drugs cruise c...,"[aryan, khan, bail, arrest, week, ago, drug, c...","[aryan, khan, bail, arrest, week, ago, drug, c...",aryan khan did not get bail today for the thir...
2,RT @ndtvfeed: Ex-National Conference Leaders D...,https://www.ndtv.com/india-news/after-resignin...,"Ex-National Conference Leaders Devender Rana, ...",a day after resigning from the national confer...,1,2.0,0.0,ndtv,0,resigning national conference devender rana he...,"[resigning, national, conference, devender, ra...","[resigning, national, conference, devender, ra...",a day after resigning from the national confer...
3,"RT @ndtvfeed: US Has Lost AI Battle To China, ...",https://www.ndtv.com/world-news/us-has-lost-ar...,"US Has Lost AI Battle To China, Pentagon's Ex-...",china has won the artificial intelligence batt...,0,5.0,0.0,ndtv,0,china artificial intelligence battle united st...,"[china, artificial, intelligence, battle, unit...","[china, artificial, intelligence, battle, unit...",china has won the artificial intelligence batt...
4,RT @ndtvfeed: A Look At Inspiring Messages On ...,https://www.ndtv.com/india-news/on-internation...,International Day Of The Girl Child 2021: Lead...,various world leaders and un agencies posted w...,3,3.0,0.0,ndtv,0,various world leaders agencies posted wishes m...,"[various, world, leader, agency, posted, wish,...","[various, world, agency, posted, wish, message...",various world leaders and un agencies posted w...


In [ ]:
df['clean_content']=""
for i in tqdm(range(len(df['clean_text_lemmatized']))):
    x=df['clean_text_lemmatized'][i]
    review=" ".join(x)
    df['clean_content'][i] = review

  0%|          | 0/2150 [00:00<?, ?it/s]/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """
100%|██████████| 2150/2150 [00:02<00:00, 963.03it/s]


In [ ]:
df.head()

,text,url,title,content,bias,retweet_count,favorite_count,source_name,source,clean_content,text_lemmatized,clean_text_lemmatized,clean_content_with_Stopwords
0,"Army Officer, 4 Others Killed In Encounter In ...",https://www.ndtv.com/india-news/army-officer-4...,"Army Officer, 4 Personnel Killed In Encounter ...",an army officer and four other personnel were ...,3,9.0,63.0,ndtv,0,army officer personnel killed action encounter...,"[army, officer, personnel, killed, action, enc...","[army, officer, personnel, killed, action, enc...",an army officer and four other personnel were ...
1,"#AryanKhan Case: No Bail For Him Today, Next H...",https://ndtv.com/india-news/mumbai-drug-bust-c...,"Aryan Khan Case: No Bail For Him Today, Next H...",aryan khan did not get bail today for the thir...,3,5.0,44.0,ndtv,0,aryan khan bail arrest week ago drug cruise ca...,"[aryan, khan, bail, arrest, week, ago, drug, c...","[aryan, khan, bail, arrest, week, ago, drug, c...",aryan khan did not get bail today for the thir...
2,RT @ndtvfeed: Ex-National Conference Leaders D...,https://www.ndtv.com/india-news/after-resignin...,"Ex-National Conference Leaders Devender Rana, ...",a day after resigning from the national confer...,1,2.0,0.0,ndtv,0,resigning national conference devender rana he...,"[resigning, national, conference, devender, ra...","[resigning, national, conference, devender, ra...",a day after resigning from the national confer...
3,"RT @ndtvfeed: US Has Lost AI Battle To China, ...",https://www.ndtv.com/world-news/us-has-lost-ar...,"US Has Lost AI Battle To China, Pentagon's Ex-...",china has won the artificial intelligence batt...,0,5.0,0.0,ndtv,0,china artificial intelligence battle united he...,"[china, artificial, intelligence, battle, unit...","[china, artificial, intelligence, battle, unit...",china has won the artificial intelligence batt...
4,RT @ndtvfeed: A Look At Inspiring Messages On ...,https://www.ndtv.com/india-news/on-internation...,International Day Of The Girl Child 2021: Lead...,various world leaders and un agencies posted w...,3,3.0,0.0,ndtv,0,various world agency posted wish message socia...,"[various, world, leader, agency, posted, wish,...","[various, world, agency, posted, wish, message...",various world leaders and un agencies posted w...


In [ ]:
# from keras.preprocessing.sequence import pad_sequences
# max_features = 30000
# maxlen = 25
# X = df['clean_text_lemmatized']
# X = pad_sequences(X, maxlen = maxlen)
# y=df['bias']

In [ ]:
for x in df['clean_text_lemmatized'][2]:
   if(x=='day'):
      print(x)

In [ ]:
for i in range(len(df)):
  for x in df['clean_text_lemmatized'][i]:
    if x=='wa':
      print(i)
      


In [ ]:
df['content'][0]

"an army officer and four other personnel were killed in action during an encounter between security forces and terrorists in jammu and kashmir's poonch district on monday, officials said. an anti-insurgency operation was launched in a village close to dera ki gali in surankote in the early hours following intelligence inputs about the presence of heavily-armed terrorists there, they said. the hiding terrorists fired on the search parties resulting in fierce gunfight and critical injuries to a junior commissioned officer or jco and four other soldiers. they died from their injuries. the encounter with the terrorists was going on and further details are awaited, a defence spokesperson said. officials said there were reports about the presence of a group of heavily-armed terrorists in the chamrer forest, after they managed to sneak from across the line of control (loc). reinforcements have been rushed to the area to plug all escape routes of the terrorists, they said. (with inputs from p

In [ ]:
#df['clean_text_lemmatized'][0]

In [ ]:
#//

In [ ]:
for i in range(len(df)):
  for x in df['clean_text_lemmatized'][i]:
    if str(x)=='said':
      print('here',i)


In [ ]:
center_with_bias_words = df.loc[df['content'].str.contains('shiv sena|shivsena|aap|bjp|congress|modi|gandhi', case=False)]
center_without_bias_words = df.loc[~df['content'].str.contains('sena|shivsena|aap|bjp|congress|modi|gandhi|minister|aam|government|aadmi', case=False)]

center_without_bias_words = center_without_bias_words.loc[center_without_bias_words['bias']==3]

center_with_bias_words = center_with_bias_words.loc[center_with_bias_words['bias']==3]
right = df.loc[df['bias']==1]
left = df.loc[df['bias']==0]
#center = df.loc[df['bias']==3]
#df2=pd.concat([left,right,center_with_bias_words])


In [ ]:
print('right',len(right))
print('left',len(left))
print('center_without_bias_words',len(center_without_bias_words))
print('center_with_bias_words',len(center_with_bias_words))



right 184
left 263
center_without_bias_words 811
center_with_bias_words 443


In [ ]:
# for xt in all_words.most_common(5000):
#   print(xt)###

In [ ]:
left.loc[left['clean_content'].str.contains('power')]#, case=False)]


,text,url,title,content,bias,retweet_count,favorite_count,source_name,source,clean_content,text_lemmatized,clean_text_lemmatized,clean_content_with_Stopwords
162,"RT @ndtvfeed: Parliament Panels Reconstituted,...",https://www.ndtv.com/india-news/parliament-pan...,"Parliament Panels Reconstituted, Congress Lead...","congress leaders shashi tharoor, anand sharma ...",0,3.0,0.0,ndtv,0,congress shashi tharoor anand sharma jairam ra...,"[congress, leader, shashi, tharoor, anand, sha...","[congress, shashi, tharoor, anand, sharma, jai...","congress leaders shashi tharoor, anand sharma ..."
197,RT @ndtvfeed: Culprits Being Given Flower Bouq...,https://www.ndtv.com/india-news/culprits-being...,"Culprits Being Given ""Flower Bouquet"": Akhiles...",attacking the uttar pradesh government over th...,0,18.0,0.0,ndtv,0,attacking uttar pradesh handling probe lakhimp...,"[attacking, uttar, pradesh, handling, probe, l...","[attacking, uttar, pradesh, handling, probe, l...",attacking the uttar pradesh government over th...


In [ ]:
right.loc[right['clean_content'].str.contains('power')]#, case=False)]

,text,url,title,content,bias,retweet_count,favorite_count,source_name,source,clean_content,text_lemmatized,clean_text_lemmatized,clean_content_with_Stopwords
503,RT @ndtvfeed: First 7-11 Store Opens In Mumbai...,https://www.ndtv.com/india-news/first-7-11-sto...,"Mukesh Ambani Brings 7-Eleven Stores To India,...","mukesh ambani, asia's richest man, will bring ...",1,22.0,0.0,ndtv,0,mukesh ambani asia richest man bring convenien...,"[mukesh, ambani, asia, richest, man, bring, co...","[mukesh, ambani, asia, richest, man, bring, co...","mukesh ambani, asia's richest man, will bring ..."
590,#Opinion: Yogi Does For Priyanka What PK Could...,https://www.ndtv.com/opinion/opinion-yogi-does...,Opinion: Yogi Does For Priyanka What PK Couldn...,it is a strange world where the man who causes...,1,100.0,791.0,ndtv,0,strange world man cause turnaround priyanka ga...,"[strange, world, man, cause, turnaround, priya...","[strange, world, man, cause, turnaround, priya...",it is a strange world where the man who causes...
1122,"Karnataka Min Ashwathnarayan lauds NEP, says '...",https://www.republicworld.com/india-news/educa...,"Karnataka Min Ashwathnarayan Lauds NEP, Says '...","karnataka minister for higher education, dr c ...",1,4.0,28.0,republictv,1,karnataka minister higher education dr c n ash...,"[karnataka, minister, higher, education, dr, c...","[karnataka, minister, higher, education, dr, c...","karnataka minister for higher education, dr c ..."
1145,Maharashtra bandh: BJP warns against forceful ...,https://www.republicworld.com/india-news/gener...,Maharashtra Bandh: BJP Warns Against Forceful ...,the maharashtra ruling alliance has called for...,1,25.0,193.0,republictv,1,maharashtra ruling alliance called statewide b...,"[maharashtra, ruling, alliance, called, statew...","[maharashtra, ruling, alliance, called, statew...",the maharashtra ruling alliance has called for...
1217,FULL INTERVIEW: Amit Shah explains Modi's 20 y...,https://www.republicworld.com/india-news/polit...,FULL INTERVIEW: Amit Shah Explains Modi's 20 Y...,hailing pm modi journey from 3-time gujarat cm...,1,7.0,59.0,republictv,1,hailing pm modi journey gujarat cm modi union ...,"[hailing, pm, modi, journey, gujarat, cm, modi...","[hailing, pm, modi, journey, gujarat, cm, modi...",hailing pm modi journey from 3-time gujarat cm...
1296,UP Min Sidharth Nath Singh slams Akhilesh Yada...,https://www.republicworld.com/india-news/polit...,A journalist was burnt alive' | UP Min Sidhart...,after former uttar pradesh chief minister and ...,1,19.0,94.0,republictv,1,uttar pradesh chief minister samajwadi party s...,"[uttar, pradesh, chief, minister, samajwadi, p...","[uttar, pradesh, chief, minister, samajwadi, p...",after former uttar pradesh chief minister and ...
1328,Well said': Smriti Irani shares powerful messa...,https://www.republicworld.com/entertainment-ne...,Well Said': Smriti Irani Shares Powerful Messa...,union minister for women and child development...,1,5.0,48.0,republictv,1,union minister woman child development smriti ...,"[union, minister, woman, child, development, s...","[union, minister, woman, child, development, s...",union minister for women and child development...


In [ ]:
left=left.head(190)

In [ ]:
center_with_bias_words=center_with_bias_words.head(200)

In [ ]:
center_without_bias_words=center_without_bias_words.head(200)


In [ ]:
print('right',len(right))
print('left',len(left))
print('center_without_bias_words',len(center_without_bias_words))
print('center_with_bias_words',len(center_with_bias_words))



right 184
left 190
center_without_bias_words 200
center_with_bias_words 200


In [ ]:
right['clean_content'][2]

'resigning national conference devender rana head jammu division party surjit singh slathia joined bjp monday rana slathia joined bjp headquarters delhi presence union minister dharmendra pradhan hardeep singh puri jitendra singh rana mla younger brother union minister jitendra singh served political advisor jammu kashmir chief minister national conference nc omar abdullah bjp general secretary charge jammu kashmir tarun chugh jammu kashmir unit chief ravinder raina welcomed party rana advocating jammu declaration joint declaration political social business organisation primarily demanding restoration statehood jammu region jammu kashmir centre revoked erstwhile jammu kashmir special status bifurcated union territory jammu kashmir ladakh'

In [ ]:
df2=pd.concat([left,right,center_without_bias_words])#,center_with_bias_words


In [ ]:
#REINDEXING
indx=[]
for i in range(len(df2)):
  indx.append(i)
df2.index=indx

In [ ]:
# ## Divide the dataset into Train and Test
# from sklearn.model_selection import train_test_split
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.4, random_state=2)

In [ ]:
######NaiiveBayes without Feature Selection
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import f1_score,classification_report,accuracy_score,confusion_matrix
from sklearn.naive_bayes import GaussianNB
## TFidf Vectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.ensemble import RandomForestClassifier

In [ ]:
X = df2['clean_content']
y=df2['bias']


In [ ]:
from nltk.tokenize import word_tokenize
all_words = []
for message in df2['clean_content']:
      words = word_tokenize(message)
      for w in words:
          all_words.append(w)
all_words = nltk.FreqDist(all_words)

In [ ]:
# print the total number of words and the 15 most common words
print('Number of words: {}'.format(len(all_words)))
print('Most common words: {}'.format(all_words.most_common(15)))

Number of words: 11190
Most common words: [('minister', 1015), ('congress', 714), ('bjp', 663), ('party', 643), ('chief', 599), ('lakhimpur', 514), ('modi', 513), ('gandhi', 494), ('union', 449), ('mishra', 430), ('kheri', 420), ('pradesh', 382), ('case', 359), ('singh', 358), ('incident', 338)]


# COMMON WORDS TESTING


In [ ]:
###TESTING
from nltk.tokenize import word_tokenize
all_words = []
for message in right['clean_content']:
      words = word_tokenize(message)
      for w in words:
          all_words.append(w)
all_words = nltk.FreqDist(all_words)
# print the total number of words and the 15 most common words
print('Number of words: {}'.format(len(all_words)))
print('Most common words: {}'.format(all_words.most_common(20)))

Number of words: 5973
Most common words: [('minister', 422), ('bjp', 356), ('modi', 314), ('party', 290), ('congress', 264), ('union', 223), ('chief', 208), ('singh', 167), ('lakhimpur', 160), ('mishra', 151), ('gandhi', 135), ('pradesh', 130), ('pm', 125), ('kheri', 117), ('incident', 106), ('national', 101), ('uttar', 98), ('crore', 96), ('home', 94), ('election', 89)]


In [ ]:
###TESTING
from nltk.tokenize import word_tokenize
all_words = []
for message in left['clean_content']:
      words = word_tokenize(message)
      for w in words:
          all_words.append(w)
all_words = nltk.FreqDist(all_words)
# print the total number of words and the 15 most common words
print('Number of words: {}'.format(len(all_words)))
print('Most common words: {}'.format(all_words.most_common(20)))

Number of words: 5108
Most common words: [('minister', 593), ('congress', 450), ('gandhi', 359), ('lakhimpur', 354), ('chief', 322), ('party', 314), ('bjp', 307), ('kheri', 303), ('mishra', 277), ('pradesh', 218), ('union', 215), ('uttar', 207), ('modi', 199), ('priyanka', 195), ('violence', 180), ('incident', 177), ('singh', 159), ('ajay', 151), ('vadra', 147), ('ashish', 145)]


In [ ]:
###TESTING
from nltk.tokenize import word_tokenize
all_words = []
for message in center_without_bias_words['clean_content']:
      words = word_tokenize(message)
      for w in words:
          all_words.append(w)
all_words = nltk.FreqDist(all_words)
# print the total number of words and the 15 most common words
print('Number of words: {}'.format(len(all_words)))
print('Most common words: {}'.format(all_words.most_common(1500)))

Number of words: 6687
Most common words: [('case', 197), ('court', 195), ('accused', 143), ('drug', 141), ('khan', 139), ('woman', 134), ('official', 127), ('arrested', 127), ('aryan', 97), ('agency', 96), ('bail', 87), ('mumbai', 82), ('friday', 76), ('high', 74), ('area', 71), ('ncb', 69), ('chief', 69), ('cruise', 68), ('told', 68), ('investigation', 66), ('covid', 66), ('according', 64), ('china', 63), ('person', 59), ('indian', 59), ('anti', 57), ('old', 57), ('added', 57), ('health', 56), ('incident', 55), ('custody', 54), ('week', 53), ('chinese', 53), ('delhi', 53), ('medium', 52), ('control', 51), ('wednesday', 51), ('bureau', 51), ('world', 51), ('saturday', 50), ('justice', 50), ('thursday', 49), ('national', 49), ('place', 48), ('member', 48), ('number', 48), ('russia', 48), ('statement', 47), ('month', 47), ('district', 46), ('company', 46), ('act', 45), ('home', 44), ('social', 44), ('air', 44), ('prize', 44), ('lawyer', 43), ('sent', 43), ('narcotic', 42), ('reported', 4

In [ ]:
#///

In [ ]:
## TFidf Vectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf_v=TfidfVectorizer(max_features=25000,ngram_range=(1,3))##35##69

In [ ]:
X=tfidf_v.fit_transform(df2['clean_content']).toarray()


In [ ]:
X

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [ ]:
## Divide the dataset into Train and Test
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33 ,random_state=42,stratify=y)

In [ ]:
count_df = pd.DataFrame(X_train, columns=tfidf_v.get_feature_names())
count_df.head()

/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


,aadmi,aadmi party,aam,aam aadmi,aam aadmi party,aap,abdullah,able,abuse,accept,...,yatra,yes,yesterday,yogi,yogi adityanath,yojana,young,younger,youth,zone
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.092476,0.0,0.064226,0.064988,0.0,0.000000,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.034724,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.0


In [ ]:
y_train.value_counts()

3    134
0    127
1    123
Name: bias, dtype: int64

In [ ]:
y_test.value_counts()


3    66
0    63
1    61
Name: bias, dtype: int64

In [ ]:
y_test.value_counts()

3    66
0    63
1    61
Name: bias, dtype: int64

In [ ]:
#///

In [ ]:
from sklearn import model_selection, naive_bayes, svm
# fit the training dataset on the NB classifier
Naive = naive_bayes.MultinomialNB()
Naive.fit(X_train,y_train)
# predict the labels on validation dataset
predictions_NB = Naive.predict(X_test)
# Use accuracy_score function to get the accuracy
print("Naive Bayes Accuracy Score -> ",accuracy_score(predictions_NB, y_test)*100)
print(classification_report(y_test,predictions_NB))

Naive Bayes Accuracy Score ->  80.52631578947368
              precision    recall  f1-score   support

           0       0.72      0.86      0.78        63
           1       0.92      0.56      0.69        61
           3       0.83      0.98      0.90        66

    accuracy                           0.81       190
   macro avg       0.82      0.80      0.79       190
weighted avg       0.82      0.81      0.80       190



In [ ]:
rf_classifier=RandomForestClassifier().fit(X_train,y_train)
prediction=rf_classifier.predict(X_test)
y.value_counts()
from sklearn.metrics import confusion_matrix,classification_report,accuracy_score
print(confusion_matrix(y_test,prediction))
print(accuracy_score(y_test,prediction))
print(classification_report(y_test,prediction))


[[44 13  6]
 [14 46  1]
 [ 0  0 66]]
0.8210526315789474
              precision    recall  f1-score   support

           0       0.76      0.70      0.73        63
           1       0.78      0.75      0.77        61
           3       0.90      1.00      0.95        66

    accuracy                           0.82       190
   macro avg       0.81      0.82      0.81       190
weighted avg       0.82      0.82      0.82       190



In [ ]:
rf_classifier=RandomForestClassifier( max_features='auto', min_samples_leaf=4, min_samples_split=5, n_estimators=1000).fit(X_train,y_train)
prediction=rf_classifier.predict(X_test)
y.value_counts()
from sklearn.metrics import confusion_matrix,classification_report,accuracy_score
print(confusion_matrix(y_test,prediction))
print(accuracy_score(y_test,prediction))
print(classification_report(y_test,prediction))


[[48  8  7]
 [20 35  6]
 [ 0  0 66]]
0.7842105263157895
              precision    recall  f1-score   support

           0       0.71      0.76      0.73        63
           1       0.81      0.57      0.67        61
           3       0.84      1.00      0.91        66

    accuracy                           0.78       190
   macro avg       0.79      0.78      0.77       190
weighted avg       0.79      0.78      0.78       190



In [ ]:
from sklearn.ensemble import RandomForestClassifier
rf_classifier=RandomForestClassifier(n_estimators=300).fit(X_train,y_train)
prediction=rf_classifier.predict(X_test)
y.value_counts()
from sklearn.metrics import confusion_matrix,classification_report,accuracy_score
print(confusion_matrix(y_test,prediction))
print(accuracy_score(y_test,prediction))
print(classification_report(y_test,prediction))


[[47  9  7]
 [19 40  2]
 [ 0  1 65]]
0.8
              precision    recall  f1-score   support

           0       0.71      0.75      0.73        63
           1       0.80      0.66      0.72        61
           3       0.88      0.98      0.93        66

    accuracy                           0.80       190
   macro avg       0.80      0.80      0.79       190
weighted avg       0.80      0.80      0.80       190



In [ ]:
//

SyntaxError: ignored

# RF  Manual Hyperparameter Tuning

In [ ]:
### Manual Hyperparameter Tuning
# {'criterion': 'entropy',
#  'max_depth': 100,
#  'max_features': 'sqrt',
#  'min_samples_leaf': 1,
#  'min_samples_split': 2,
#  'n_estimators': 600}
model=RandomForestClassifier(n_estimators=900,criterion='entropy',
                             max_features='sqrt',min_samples_leaf=1,min_samples_split=2,random_state=100).fit(X_train,y_train)
predictions=model.predict(X_test)
print(confusion_matrix(y_test,predictions))
print(accuracy_score(y_test,predictions))
print(classification_report(y_test,predictions))

In [ ]:
//

#RF WITH RANDOMSEARCH CV

RF WITH RANDOMSEARCH CV

In [ ]:
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import RandomizedSearchCV
# Number of trees in random forest
n_estimators = [int(x) for x in np.linspace(start = 200, stop = 2000, num = 10)]
# Number of features to consider at every split
max_features = ['auto', 'sqrt','log2']
# Maximum number of levels in tree
max_depth = [int(x) for x in np.linspace(50, 500,10)]
# Minimum number of samples required to split a node
min_samples_split = [2, 5, 10,14]
# Minimum number of samples required at each leaf node
min_samples_leaf = [1, 2, 4,6,8]
# Create the random grid
random_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
              'criterion':['entropy','gini']}
print(random_grid)

In [ ]:
rf=RandomForestClassifier()
rf_randomcv=RandomizedSearchCV(estimator=rf,param_distributions=random_grid,n_iter=20,cv=3,verbose=2,
                               random_state=50,n_jobs=-1)
### fit the randomized model
rf_randomcv.fit(X_train,y_train)

In [ ]:
rf_randomcv.best_params_

In [ ]:
best_random_grid=rf_randomcv.best_estimator_
from sklearn.metrics import accuracy_score
y_pred=best_random_grid.predict(X_test)
print(confusion_matrix(y_test,y_pred))
print("Accuracy Score {}".format(accuracy_score(y_test,y_pred)))
print("Classification report: \n {}".format(classification_report(y_test,y_pred)))

In [ ]:
//

#RF WITH GridSearchCV


RF WITH GridSearchCV


In [ ]:
from sklearn.model_selection import GridSearchCV

param_grid = {
    'criterion': [rf_randomcv.best_params_['criterion']],
    'max_depth': [rf_randomcv.best_params_['max_depth']],
    'max_features': [rf_randomcv.best_params_['max_features']],
    'min_samples_leaf': [rf_randomcv.best_params_['min_samples_leaf'], 
                         rf_randomcv.best_params_['min_samples_leaf']+2, 
                         rf_randomcv.best_params_['min_samples_leaf'] + 4],
    'min_samples_split': [rf_randomcv.best_params_['min_samples_split'] - 2,
                          rf_randomcv.best_params_['min_samples_split'] - 1,
                          rf_randomcv.best_params_['min_samples_split'], 
                          rf_randomcv.best_params_['min_samples_split'] +1,
                          rf_randomcv.best_params_['min_samples_split'] + 2],
    'n_estimators': [rf_randomcv.best_params_['n_estimators'] - 200, rf_randomcv.best_params_['n_estimators'] - 100, 
                     rf_randomcv.best_params_['n_estimators'], 
                     rf_randomcv.best_params_['n_estimators'] + 100, rf_randomcv.best_params_['n_estimators'] + 200]
}

print(param_grid)
#### Fit the grid_search to the data
rf=RandomForestClassifier()
grid_search=GridSearchCV(estimator=rf,param_grid=param_grid,cv=10,n_jobs=-1,verbose=2)
grid_search.fit(X_train,y_train)


In [ ]:
//

# RF WITH TPOT OPTIMIZATION

In [ ]:
pip install tpot


In [ ]:
from tpot import TPOTClassifier

tpot_classifier = TPOTClassifier(generations= 3, population_size= 30, offspring_size= 10,
                                 verbosity= 2, early_stop= 12,
                                 config_dict={'sklearn.ensemble.RandomForestClassifier':param}, 
                                 cv = 4, scoring = ['accuracy','recall','precision'])
tpot_classifier.fit(X_train,y_train)

In [ ]:
### Manual Hyperparameter Tuning
# {'criterion': 'entropy',
#  'max_depth': 100,
#  'max_features': 'sqrt',
#  'min_samples_leaf': 1,
#  'min_samples_split': 2,
#  'n_estimators': 600}
#model=RandomForestClassifier( criterion='gini', max_depth=340, max_features='auto', min_samples_leaf=2, min_samples_split=5, n_estimators=2000).fit(X_train,y_train)
model=RandomForestClassifier(criterion='entropy', max_depth=780, max_features='auto', min_samples_leaf=4, min_samples_split=5, n_estimators=1000).fit(X_train,y_train)
predictions=model.predict(X_test)
print(confusion_matrix(y_test,predictions))
print(accuracy_score(y_test,predictions))
print(classification_report(y_test,predictions))

In [ ]:
pip install optuna


In [ ]:
import optuna
import sklearn.svm
def objective(trial):

    classifier = trial.suggest_categorical('classifier', ['RandomForest', 'SVC'])
    
    if classifier == 'RandomForest':
        n_estimators = trial.suggest_int('n_estimators', 200, 2000,10)
        max_depth = int(trial.suggest_float('max_depth', 10, 100, log=True))

        clf = sklearn.ensemble.RandomForestClassifier(
            n_estimators=n_estimators, max_depth=max_depth)
    else:
        c = trial.suggest_float('svc_c', 1e-10, 1e10, log=True)
        
        clf = sklearn.svm.SVC(C=c, gamma='auto')

    return sklearn.model_selection.cross_val_score(
        clf,X_train,y_train, n_jobs=-1, cv=3).mean()

In [ ]:
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=100)

trial = study.best_trial

print('Accuracy: {}'.format(trial.value))
print("Best hyperparameters: {}".format(trial.params))

In [ ]:
rf=RandomForestClassifier(n_estimators=330,max_depth=30)
rf.fit(X_train,y_train)

In [ ]:
y_pred=rf.predict(X_test)
print(confusion_matrix(y_test,y_pred))
print(accuracy_score(y_test,y_pred))
print(classification_report(y_test,y_pred))

In [ ]:
from sklearn.naive_bayes import MultinomialNB
clf = MultinomialNB().fit(X_train, y_train)

In [ ]:
from sklearn.pipeline import Pipeline
text_clf = Pipeline([('vect', CountVectorizer()),
                     ('tfidf', TfidfTransformer()),
                     ('clf', MultinomialNB())])
text_clf = text_clf.fit(X_train, y_train)

In [ ]:
from sklearn.metrics import accuracy_score
y_pred=text_clf.predict(X_test)
print(confusion_matrix(y_test,y_pred))
print("Accuracy Score {}".format(accuracy_score(y_test,y_pred)))
print("Classification report: \n {}".format(classification_report(y_test,y_pred)))

# New Section